<a href="https://colab.research.google.com/github/mechhector/visao_computacional20212/blob/main/trabalho4/leaf_classification_dip_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# HU Moments

def HU_FE(image):
        moments = cv2.moments(image.astype(np.float64))
        return np.asarray(cv2.HuMoments(moments).flatten())


# LBP

def LBP_FE(image):
        lbp_image = local_binary_pattern(image, 256, 1, "uniform")
        return np.histogram(lbp_image.ravel(), bins=256)
        return (lbp_image)

# Gray Level Cooccurrency Matrix

def GLCM_FE(image):
        glcm = greycomatrix(image, [1], [0], 256, symmetric=True, normed=True)
        xs = []
        xs.append(greycoprops(glcm, 'dissimilarity')[0, 0])
        xs.append(greycoprops(glcm, 'correlation')[0, 0])
        xs.append(greycoprops(glcm, 'homogeneity')[0, 0])
        xs.append(greycoprops(glcm, 'ASM')[0, 0])
        xs.append(greycoprops(glcm, 'energy')[0, 0])
        return np.asarray(xs);


In [ ]:
def normalizeImage(v):
  v = (v - v.min()) / (v.max() - v.min())
  result = (v * 255).astype(np.uint8)

  #cv2_imshow(result)

  return result

In [ ]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import time

from scipy import stats
from scipy import ndimage

from skimage.feature import greycomatrix, greycoprops, local_binary_pattern
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, cross_val_score, StratifiedShuffleSplit
from sklearn.feature_extraction import image
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB

from google.colab.patches import cv2_imshow

In [ ]:
# features split

dataset_features = np.zeros((340,7))

for i in range(a.shape[0]):
  
  b = a[i]

  dataset_features[i] = b[9:]

print(dataset_features)

#np.savetxt('texture_features.txt', dataset_features, delimiter=',')

In [ ]:
# Dataset shape and texture features

shape_features = np.loadtxt('/content/drive/MyDrive/Computer_Vision/Paper/shape_reatures.txt', delimiter = ',')

texture_features = np.loadtxt('/content/drive/MyDrive/Computer_Vision/Paper/texture_features.txt', delimiter = ',')

glcm_shape_features = np.loadtxt('/content/drive/MyDrive/Computer_Vision/Paper/merged_glcm_shape.txt', delimiter =',' )

glcm_hu_features = np.loadtxt('/content/drive/MyDrive/Computer_Vision/Paper/glcm_hu_features.txt', delimiter = ',')

dataset_features = np.loadtxt('/content/drive/MyDrive/Computer_Vision/Paper/dataset_features.txt', delimiter = ',')

print(glcm_shape_features.shape)

(340, 12)


In [ ]:
# Merged Leaf Matrix

leaf_class = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36]
numberOfSamples = [12,10,10,8,12,8,10,11,14,13,16,12,13,12,10,12,11,13,9,12,11,12,12,12,11,11,11,11,11,10]
targets_vector = []

merged_leaf_features = np.zeros((340,12))

line_index = 0
count = 1

for i in leaf_class:
  for j in range(numberOfSamples[leaf_class.index(i)]):
  
    #print(count)
    #print(line_index)

    hold_j = numberOfSamples[leaf_class.index(i)] + 1

    
    if i > 9 and j+1 > 9:

      data_path = '/content/drive/MyDrive/Computer_Vision/Paper/merged_leaf/merged_leaf_C{}_EX{}.JPG'


    elif i <= 9 and j+1 > 9:

      data_path = '/content/drive/MyDrive/Computer_Vision/Paper/merged_leaf/merged_leaf_C0{}_EX{}.JPG'


    elif i > 9 and j+1 <= 9:

      data_path = '/content/drive/MyDrive/Computer_Vision/Paper/merged_leaf/merged_leaf_C{}_EX0{}.JPG'
    
    else:

      data_path = '/content/drive/MyDrive/Computer_Vision/Paper/merged_leaf/merged_leaf_C0{}_EX0{}.JPG'


    # Feature Extraction

    #print(data_path.format(i,j+1))

    if line_index == 45:
      print(data_path.format(i,j+1))


    leaf_image = cv2.imread(data_path.format(i,j+1),0)

    if type(leaf_image) == None:
      print(data_path.format(i,j+1))


    glcm_leaf_image = GLCM_FE(leaf_image)

    
    hu_leaf_image = HU_FE(leaf_image)

    merged_features = np.append(glcm_leaf_image,hu_leaf_image)

    if line_index < 340:

      merged_leaf_features[line_index] = merged_features

    count = count + 1
    
    line_index = line_index + 1

    targets_vector.append(i)


print(merged_leaf_features.shape)

print(targets_vector)

np.savetxt('glcm_hu_features.txt', merged_leaf_features, delimiter=',')


/content/drive/MyDrive/Computer_Vision/Paper/merged_leaf/merged_leaf_C05_EX06.JPG
(340, 12)
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28

In [ ]:
start_time = time.time()
#main()

data_list = [texture_features, shape_features, dataset_features, glcm_shape_features, glcm_hu_features]
data_name =  ['texture_features', 'shape_features', 'dataset_features', 'glcm_shape_features', 'glcm_hu_features']
hd = 0 
#clf = KNeighborsClassifier(n_neighbors=5)
#clf = MLPClassifier(random_state=1, max_iter=10000)
#clf = tree.DecisionTreeClassifier()
#clf = GaussianNB()

for p in data_list: 

  X = p
  y = np.asarray(targets_vector)

  sss = StratifiedShuffleSplit(n_splits=10 , test_size=0.1)

  for train_index , test_index in sss.split(X,y): 
    X_train , X_test = X[train_index] , X[test_index]
    y_train , y_test = y[train_index] , y[test_index]



    # Acc, std deviation and confusion matrix

  std = []

  mean_acc = 0

  c_matrix = 0

  for k, (train, test) in enumerate(sss.split(X, y)):
      clf.fit(X[train], y[train])

      predicted = clf.predict(X_test)

      acc = metrics.accuracy_score(y_test, predicted)

      std.append(acc)

      mean_acc = mean_acc + acc

      c_matrix = c_matrix + confusion_matrix(y_test, predicted)

      #print(confusion_matrix(y_test, predicted))
      #print(confusion_matrix(y_test, predicted).sum())

      #print("Accuracy: " f"{metrics.accuracy_score(y_test, predicted)}\n")

  mean_c_matrix = c_matrix/10
  for i in range(mean_c_matrix.shape[0]):
    for j in range(mean_c_matrix.shape[1]):
      c_matrix[i][j] = round(c_matrix[i][j])    

  #print('mean confusion matrix:') 
  #print(c_matrix/10) 

  
  print(data_name[hd], str(clf))   
 
  print('mean accuracy: ' , mean_acc/10) 

  #print('test samples: ' , c_matrix.sum()/10)

  print('std deviation: ' , np.asarray(std).std())
  print('\n')
  

  hd = hd + 1

# To plot the confusion matrix

print("--- %s seconds ---" % (time.time() - start_time))

'''
plt.figure(figsize=(11,7))
sn.set(font_scale=1.8) 
sn.heatmap(c_matrix/10, annot=True, annot_kws={"size": 18}) # font size
plt.suptitle("Matriz de Confusão")
plt.xlabel("Predito")
plt.ylabel("Real")
plt.gcf().set_dpi(50)
plt.show()
'''